### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [4]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [5]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [6]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [7]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [8]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [9]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [10]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [11]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [12]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [13]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [14]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [15]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [16]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  4703, 10870,  4333])

In [17]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [18]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [19]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [20]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [21]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [22]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


##Punto 1)

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import random

# Seleccionar 5 documentos al azar
random.seed(42)
sample_indices = random.sample(range(X_train.shape[0]), 5)

for idx in sample_indices:
    # Similaridad coseno con el resto de documentos
    cossim = cosine_similarity(X_train[idx], X_train)[0]

    # Obtener los 5 documentos más similares (excluyendo el seleccionado)
    most_similar_docs = np.argsort(cossim)[::-1][1:6]

    # Imprimir el documento original y su clase
    print(f"\nDocumento seleccionado (ID={idx}):")
    print(newsgroups_train.data[idx][:300], "...")  # Primeros caracteres
    print(f"\n CLASE: {newsgroups_train.target_names[y_train[idx]]}")

    # Imprimir los documentos más similares y sus clases
    print("\n")
    print("\n5 Documentos más similares:")
    for i in most_similar_docs:
        print(f"\nDocumento ID={i}")
        print(newsgroups_train.data[i][:300], "...")  # Primeros caracteres
        print(f"CLASE: {newsgroups_train.target_names[y_train[i]]}")

    # Similitud de clase
    similar_classes = [y_train[i] for i in most_similar_docs]
    print(f"\nClases de los documentos más similares: {similar_classes}")
    if y_train[idx] in similar_classes:
        print("-> La similitud parece consistente con la clase del documento.")
    else:
        print("-> La similitud no parece consistente con la clase del documento.")



Documento seleccionado (ID=10476):
This is a general question for US readers:

How extensive is the playoff coverage down there?  In Canada, it is almost
impossible not to watch a series on TV (ie the only two series I have not had
an opportunity to watch this year are Wash-NYI and Chi-Stl, the latter because
I'm in the wrong time zo ...

 CLASE: rec.sport.hockey



5 Documentos más similares:

Documento ID=5064

I only have one comment on this:  You call this a *classic* playoff year
and yet you don't include a Chicago-Detroit series.  C'mon, I'm a Boston
fan and I even realize that Chicago-Detroit games are THE most exciting
games to watch. ...
CLASE: rec.sport.hockey

Documento ID=9623
Accounts of Anti-Armenian Human Right Violations in Azerbaijan #012
                 Prelude to Current Events in Nagorno-Karabakh

        +---------------------------------------------------------+
        |                                                         |
        |  I saw a naked girl wi 

## Punto 2)
Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score, make_scorer
from sklearn.datasets import fetch_20newsgroups

# Definir el F1-score macro como métrica de evaluación
f1_macro = make_scorer(f1_score, average='macro')

# Configurar un pipeline para realizar el preprocesamiento y clasificación
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

# Definir los parámetros a ajustar para el vectorizador y los modelos
param_grid = {
    'tfidf__min_df': [1, 5, 10],  # frecuencia mínima de término
    'tfidf__max_df': [0.7, 0.85, 0.95],  # frecuencia máxima de término
    'tfidf__ngram_range': [(1, 1), (1, 2)],  # rango de n-gramas
    'tfidf__norm': ['l1', 'l2'],  # normalización de los vectores
    'clf': [MultinomialNB(), ComplementNB()],
    'clf__alpha': [0.1, 0.5, 1.0]  # parámetro de suavización
}

# Configuro la búsqueda aleatoria
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_grid,
    n_iter=30,
    cv=5,
    scoring=f1_macro,
    random_state=42,
    n_jobs=1,
    verbose=2
)

# Entrenar el modelo usando GridSearchCV
random_search.fit(X_train, y_train)

# Mostrar los mejores parámetros y el mejor puntaje de F1-score macro
print("Mejores parámetros encontrados:")
print(random_search.best_params_)
print(f"Mejor F1-score macro en conjunto de validación cruzada: {random_search.best_score_:.4f}")

# Evaluar el mejor modelo en el conjunto de test
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
test_f1_macro = f1_score(y_test, y_pred, average='macro')
print(f"F1-score macro en conjunto de test: {test_f1_macro:.4f}")


Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] END clf=ComplementNB(), clf__alpha=1.0, tfidf__max_df=0.85, tfidf__min_df=10, tfidf__ngram_range=(1, 1), tfidf__norm=l2; total time=   1.9s
[CV] END clf=ComplementNB(), clf__alpha=1.0, tfidf__max_df=0.85, tfidf__min_df=10, tfidf__ngram_range=(1, 1), tfidf__norm=l2; total time=   1.6s
[CV] END clf=ComplementNB(), clf__alpha=1.0, tfidf__max_df=0.85, tfidf__min_df=10, tfidf__ngram_range=(1, 1), tfidf__norm=l2; total time=   1.6s
[CV] END clf=ComplementNB(), clf__alpha=1.0, tfidf__max_df=0.85, tfidf__min_df=10, tfidf__ngram_range=(1, 1), tfidf__norm=l2; total time=   1.6s
[CV] END clf=ComplementNB(), clf__alpha=1.0, tfidf__max_df=0.85, tfidf__min_df=10, tfidf__ngram_range=(1, 1), tfidf__norm=l2; total time=   1.6s
[CV] END clf=ComplementNB(), clf__alpha=1.0, tfidf__max_df=0.95, tfidf__min_df=10, tfidf__ngram_range=(1, 1), tfidf__norm=l2; total time=   2.1s
[CV] END clf=ComplementNB(), clf__alpha=1.0, tfidf__max_df=0.95, tfi

## Punto 3)
Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras. Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente".

In [41]:
tfidfvect = TfidfVectorizer(
    stop_words='english',
    max_df=0.8,  # términos que aparecen en más del 80% de los documentos
    ngram_range=(1, 1),
    min_df=5,    # términos que aparecen en menos de 5 documentos
    max_features=10000  # 10000 palabras más relevantes
)

# Vectorización
X_train = tfidfvect.fit_transform(newsgroups_train.data)
idx2word = {v: k for k, v in tfidfvect.vocabulary_.items()}

# Selección de palabras
words = ['politics', 'computer', 'health', 'space', 'religion']
index_words = [tfidfvect.vocabulary_[word] for word in words if palabra in tfidfvect.vocabulary_]

# Mostrar similitudes de las palabras seleccionadas
for idx in index_words:
    cossim_palabra = cosine_similarity(X_train.T[idx], X_train.T).flatten()
    similares = np.argsort(cossim_palabra)[::-1][1:6]
    palabra = idx2word[idx]
    palabras_similares = [idx2word[i] for i in similares]

    print(f"\nPalabra: {palabra}")
    print("Palabras más similares:", palabras_similares)



Palabra: politics
Palabras más similares: ['talk', 'renewal', 'practiced', 'guns', 'fascist']

Palabra: computer
Palabras más similares: ['wu', 'graphics', 'drive', 'incremental', 'reboot']

Palabra: health
Palabras más similares: ['care', 'insurance', 'medical', 'medicine', 'province']

Palabra: space
Palabras más similares: ['nasa', 'shuttle', 'exploration', 'aeronautics', 'launch']

Palabra: religion
Palabras más similares: ['religious', 'religions', 'christianity', 'christian', 'wars']
